In [ ]:
###################################################
#################### Problem 4 ################
###################################################
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
import IPython.display
from IPython.display import display
import os
import re
import logging



def solve_ricatti_equations(A,B,Q,R,horizon_length):
    """
    This function solves the backward Riccatti equations for regulator problems of the form
    min xQx + sum(xQx + uRu) subject to xn+1 = Axn + Bun
    
    Arguments:
    A, B, Q, R: numpy arrays defining the problem
    horizon_length: length of the horizon
    
    Returns:
    P: list of numpy arrays containing Pn from N to 0
    K: list of numpy arrays containing Kn from N-1 to 0
    """
    
    K = []#will contain the list of Ks from N-1 to 0
    P = []#will contain the list of Ps from N to 0
    
    #initial condition
    P_N = Q

    #C = (B.T*P_Njiayi*B+R).I
    #initial condition
    
    
    #P.append(Q+(A.T * P_Njiayi * A) - (A.T * P_Njiayi *B)*K_N)

    for i in range(horizon_length):
        P_N = Q + A.T*P_N*A - A*P_N*B*(B.T*P_N*B+R).I*B.T*P_N*A
        K_N = -(B.T*P_N*B+R).I*B.T*P_N*A
        P.append(P_N)
        K.append(K_N)

    return P[-1], K[-1]   
    
    
    '''
    P_Njiayi = Q
    K_N = 2
    #K_N = np.matrix('[-0.26008493; -0.69922151; -0.77565464]')
    

    #C = (B.T*P_Njiayi*B+R).I
    
    
    P.append(Q+(A.T * P[len(P)-1] * A) - (A.T * P[len(P)-1]*B)*K_N)
    
    for i in range(horizon_length):
        
        K.append(-((B.T*P[len(P)-1]*B+R).I)*B.T*P[len(P)-1]*A)

        P.append( Q+(A.T * P[len(P)-1] * A) + (A.T * P[len(P)-1] *B)*K[len(K)-1])

    return P[::-1],K[::-1]   
    '''

if __name__ == "__main__":

    A = np.matrix('[0.1 0.2 0.3;0.2 0.5 0.9;0.3 0.1 0.7]')
    B = np.matrix('[1.0;0.6;0.1]')
    Q = np.matrix('[1.0 0.5 0.5;0.1 0.6 0.2;0.9 0.6 0.2]')
    R = np.matrix('[0.6]')
    solution = solve_ricatti_equations(A,B,Q,R,100)
    print('P = ',solution[0])
    print('K = ',solution[1])




In [ ]:
def simulate_cart_pole(x0, K, uff, horizon_length, mp=1., mc=5., l=1., g=9.81):
    """
    This function integrates the cart-pole system (the nonlinear system) for horizon_length steps
    
    Arguments:
    x0: numpy vector, initial value for the system (4 numbers for x,v,theta,omega)
    K: a list of control gains (of length horizon_length)
    uff: a list of feedforward control inputs such that the control will be f[i]=K[i] * x[i] + uff[i]
    horizon_length: lenght of the horizon to integrate
    
    Returns:
    x a numpy array containing the integration result
    u a numpy array containing the control at each step
    """
    delta_t = 0.01
        
    x=np.empty([4, horizon_length+1])
    x[:,0] = x0
    
    u=np.empty([horizon_length])

    for i in range(horizon_length):
        u[i] = K[i].dot(x[:,i]) + uff[i]
        dx = np.array([x[1,i],
                      (u[i] + mp*np.sin(x[2,i])*(l*(x[3,i]**2) + g * np.cos(x[2,i])))/(mc+mp*np.sin(x[2,i])**2),
                      x[3,i],
                      (-u[i]*np.cos(x[2,i])-mp*l*(x[3,i]**2)*np.cos(x[2,i])*np.sin(x[2,i])-(mp+mc)*g*np.sin([x[2,i]]))/(l*(mc+mp*np.sin(x[2,i])**2))
                       ])
        x[:,i+1] = x[:,i] + delta_t * dx
    return x, u